# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [1]:
import json

In [2]:
with open("courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [3]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

there are 7813 courses in the dataset
here is the data for course 1246


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ['An', 'Huang', 'anhuang@brandeis.edu'],
 'coinstructors': [],
 'code': ['MATH', '223A'],
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [4]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])

In [5]:
print('notice that the instructor and code are tuples now')
courses[1246]

notice that the instructor and code are tuples now


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ('An', 'Huang', 'anhuang@brandeis.edu'),
 'coinstructors': (),
 'code': ('MATH', '223A'),
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to asnwer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

In [6]:
## a: how many faculty taught COSI courses last year?
len([i for i in courses if i['subject'] == "COSI" ])

272

In [6]:
## b: what is the total number of students taking COSI courses last year?
def filterItemByGroup(subject, item, group = 'subject'):
    return [i[item] for i in courses if i[group] == subject]
sum(filterItemByGroup("COSI", "enrolled"))

2223

In [8]:
## c: what was the median size of a COSI course last year (counting only those courses with at least 10 students)
from numpy import median
median([i for i in filterItemByGroup("COSI", "enrolled") if i >= 10])

37.0

In [9]:
## d: create a list of tuples (E,S) where S is a subject and E is the number of students enrolled in courses in that subject, sort it and print the top 10. This shows the top 10 subjects in terms of number of students taught.
subjects = set([course['subject'] for course in courses])
## this function sort the input tuple by counts and output the top n rows where n is assigned by the second parameter
def topNFromTuple(tupleIn, n = 10):
    subTuples_sorted = sorted(tupleIn, key = lambda x : x[1], reverse = True)
    return subTuples_sorted[0:n]

subTuples = [(i, sum(filterItemByGroup(i, "enrolled"))) for i in subjects]
topNFromTuple(subTuples)

[('HS', 5318),
 ('BIOL', 3085),
 ('BUS', 2766),
 ('HWL', 2734),
 ('CHEM', 2322),
 ('ECON', 2315),
 ('COSI', 2223),
 ('MATH', 1785),
 ('PSYC', 1704),
 ('ANTH', 1144)]

In [10]:
## e: do the same as in (d) but print the top 10 subjects in terms of number of courses offered
subTuples = [(i, len(set(filterItemByGroup(i, "code")))) for i in subjects] # use set to remove duplicates in a list of "code"s
topNFromTuple(subTuples)

[('HS', 188),
 ('MUS', 72),
 ('BIOL', 72),
 ('ENG', 71),
 ('ANTH', 68),
 ('BUS', 65),
 ('NEJS', 64),
 ('PSYC', 57),
 ('MATH', 55),
 ('ECON', 53)]

In [13]:
## f: do the same as (d) but print the top 10 subjects in terms of number of faculty teaching courses in that subject
subTuples = [(i, len(set(filterItemByGroup(i, "instructor")))) for i in subjects] # set to remove duplicates in a list of instructors
topNFromTuple(subTuples)

[('HS', 87),
 ('BIOL', 67),
 ('ECON', 52),
 ('BCHM', 49),
 ('BUS', 47),
 ('HIST', 47),
 ('BCBP', 46),
 ('HWL', 42),
 ('MATH', 37),
 ('NEJS', 37)]

In [14]:
## g: list the top 20 faculty in terms of number of students they taught
## Note: a student enrolled in 2 courses will be counted as 2 for a instrcutor, as there is no way to distinguish them!
instructors = set([course['instructor'] for course in courses])
subTuples = [(i, sum(filterItemByGroup(i, "enrolled", group = "instructor"))) for i in instructors]
topNFromTuple(subTuples, 20)

[(('Leah', 'Berkenwald', 'leahb@brandeis.edu'), 926),
 (('Kene Nathan', 'Piasta', 'kpiasta@brandeis.edu'), 583),
 (('Stephanie', 'Murray', 'murray@brandeis.edu'), 515),
 (('Milos', 'Dolnik', 'dolnik@brandeis.edu'), 489),
 (('Maria', 'de Boef Miara', 'mmiara@brandeis.edu'), 450),
 (('Bryan', 'Ingoglia', 'ingoglia@brandeis.edu'), 439),
 (('Rachel V.E.', 'Woodruff', 'woodruff@brandeis.edu'), 422),
 (('Timothy J', 'Hickey', 'tjhickey@brandeis.edu'), 411),
 (('Daniel', 'Breen', 'dbreen91@brandeis.edu'), 375),
 (('Melissa', 'Kosinski-Collins', 'kosinski@brandeis.edu'), 365),
 (('Claudia', 'Novack', 'novack@brandeis.edu'), 355),
 (('Antonella', 'DiLillo', 'dilant@brandeis.edu'), 342),
 (('Jon', 'Chilingerian', 'chilinge@brandeis.edu'), 330),
 (('Ahmad', 'Namini', 'anamini@brandeis.edu'), 327),
 (('Iraklis', 'Tsekourakis', 'tsekourakis@brandeis.edu'), 316),
 (('Geoffrey', 'Clarke', 'geoffclarke@brandeis.edu'), 315),
 (('Peter', 'Mistark', 'pmistark@brandeis.edu'), 277),
 (('Brenda', 'Anderson'

In [15]:
## h: list the top 20 courses in terms of number of students taking that course (where you combine different sections and semesters, i.e. just use the subject and course number)
## Note: here I just use the subject and course number!!
codes = set([course['code'] for course in courses])
subTuples = [(i, sum(filterItemByGroup(i, "enrolled", group = "code"))) for i in codes]
topNFromTuple(subTuples, 20)

[(('HWL', '1'), 940),
 (('HWL', '1-PRE'), 879),
 (('BIOL', '14A'), 358),
 (('COSI', '10A'), 343),
 (('PSYC', '10A'), 336),
 (('BIOL', '15B'), 287),
 (('MATH', '10A'), 280),
 (('BIOL', '18B'), 274),
 (('BIOL', '18A'), 262),
 (('CHEM', '29A'), 245),
 (('CHEM', '29B'), 239),
 (('CHEM', '25A'), 236),
 (('PSYC', '51A'), 231),
 (('CHEM', '25B'), 226),
 (('COSI', '12B'), 225),
 (('BUS', '6A'), 215),
 (('CHEM', '18A'), 208),
 (('ECON', '10A'), 207),
 (('MATH', '15A'), 204),
 (('ANTH', '1A'), 201)]

In [16]:
# i: Create your own interesting question (each team member creates their own) and use Python to answer that question.
# zihao: who is the instructor involved in teaching courses in most subjects?
import pandas as pd
subdata = [[i['instructor'], i['subject']] for i in courses]
df = pd.DataFrame(subdata, columns = ['instructor', 'subject'])

df_filter = df.groupby('instructor')['subject'].apply(lambda x: len(set(x)))
df_filter.sort_values(ascending = False)

df[df['instructor'] == df_filter.sort_values(ascending = False).index[1]].drop_duplicates()

,instructor,subject
573,"(David, Powelstock, pstock@brandeis.edu)",HIST
1898,"(David, Powelstock, pstock@brandeis.edu)",COML
2039,"(David, Powelstock, pstock@brandeis.edu)",ENG
2074,"(David, Powelstock, pstock@brandeis.edu)",ECS
2531,"(David, Powelstock, pstock@brandeis.edu)",RUS
3453,"(David, Powelstock, pstock@brandeis.edu)",RECS
